In [1]:
import random
from gophish import Gophish
from gophish.models import SMTP, Page, Template, Campaign, User, Group
import urllib3
import json

In [ ]:
config_file = "config.json"

with open(config_file, "r") as file:
    config = json.load(file)
    GOPHISH_API_KEY = config.get("GOPHISH_API_KEY")

In [4]:
phish_api = Gophish(GOPHISH_API_KEY, verify=False)

In [6]:
# SMTP Sending Profile Data
sending_profile = SMTP(
    name="Training Sending Profile",
    from_address="aleksander.szostakowski@gmail.com",  # Replace with your email address
    from_name="Cybersecurity Trainer",
    reply_to_address="aleksander.szostakowski@gmail.com",  # Replace with your email address
    host="smtp.gmail.com:587",  # Replace with your SMTP server host
    username="aleksander.szostakowski@gmail.com",  # Replace with your SMTP username
    password="",  # Replace with your SMTP password
    port=587,  # SMTP port (use 465 for SSL, 587 for TLS)
    tls=True,  # Use TLS if needed
    ssl=False,  # Use SSL if needed
)


In [7]:
type(sending_profile)

gophish.models.SMTP

In [11]:
smtp = phish_api.smtp.post(sending_profile)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Error: SMTP name already in use

In [22]:
phish_api.smtp.get(2).name

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


'Training Sending Profile'

In [19]:
test = SMTP(id=2)

In [38]:
dummy_html = "<html><head><title></title></head><body><h1>Click here</h1><p><a href=''>Faaaaaast</a></p></body></html>"

In [42]:
dummy_landing = f'<html><head>	<title></title></head><body><p><b>Stuuuupid</b></p><form action=""><label for="email">Email:</label> <input id="email" name="email" type="email"/><br/><br/><label for="pwd">Password:</label> <input id="pwd" minlength="8" name="password" type="password"/><br/><br/><input type="submit"/></form></body></html>'

In [ ]:
group_2 = Group(name='Example Gr', targets=[User(first_name='Alek', last_name='Szostakowski', email='aleksander.szostakowski@gmail.com', posistion='data janitor')])

In [51]:
campaign = Campaign(
    name="Cybersecurity Awareness Scam",
    template = Template(name="self"),
    page = Page(name="self"),
    smtp=smtp,
    groups=[Group(name="self")]
)

In [52]:
phish_api.campaigns.post(campaign)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [54]:
campaign.__dict__

{'id': None,
 'name': 'Cybersecurity Awareness Scam',
 'created_date': datetime.datetime(2025, 1, 15, 14, 20, 27, 411353, tzinfo=tzlocal()),
 'launch_date': datetime.datetime(2025, 1, 15, 14, 20, 27, 411353, tzinfo=tzlocal()),
 'send_by_date': None,
 'completed_date': None,
 'template': <gophish.models.Template at 0x199bd1435d0>,
 'page': <gophish.models.Page at 0x199bd179e90>,
 'results': [],
 'status': None,
 'timeline': [],
 'smtp': <gophish.models.SMTP at 0x199bc5a49d0>,
 'url': None,
 'groups': [{'name': 'self',
   'modified_date': '2025-01-15T14:20:27.411353+01:00'}]}

In [62]:
camp = phish_api.campaigns.get(3)
camp.__dict__

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'id': 3,
 'name': 'Cybersecurity Awareness Scam',
 'created_date': '2025-01-15T14:58:31.8999001Z',
 'launch_date': '2025-01-15T13:20:27.411353Z',
 'send_by_date': '0001-01-01T00:00:00Z',
 'completed_date': '0001-01-01T00:00:00Z',
 'template': <gophish.models.Template at 0x199bd2ffb50>,
 'page': <gophish.models.Page at 0x199bd179ed0>,
 'results': [<gophish.models.Result at 0x199bd2fda10>],
 'status': 'In progress',
 'timeline': [<gophish.models.TimelineEntry at 0x199bd2fc8d0>,
 'smtp': <gophish.models.SMTP at 0x199bd2ff5d0>,
 'url': '',
 'groups': []}

In [65]:
for x in camp.results:
    print(x.__dict__)

{'id': 'gVZGtHU', 'first_name': 'Aleksander', 'last_name': 'Szostakowski', 'email': 'aleksander.szostakowski@gmail.com', 'position': 'big boss', 'ip': '', 'latitude': 0, 'longitude': 0, 'status': 'Email Sent'}
